In [3]:
# Library
import pandas as pd

# Data Loading
data = pd.read_csv("./data/rms_df.csv")

# 필요없는 column 제거
data = data[data.columns[1:]]

# Top 30
TOP_30_DATA = data.sort_values(by="percent of rms")[0:30]

# Bottom 30
BOTTOM_30_DATA = data.sort_values(by="percent of rms", ascending=False)[0:30]

# Merge Data
data = pd.read_csv("./data/df_final.csv")

# 필요없는 column 제거
data = data[data.columns[1:]]

# Date를 익덱스로 변경
data["date"] = pd.to_datetime(data["date"])
data = data.set_index("date")

# OHLC와 code 추출
data = data[["open", "high", "low", "close", "code"]]

# 종목 LIST
TOP_LIST = TOP_30_DATA["code_number"].values
BOTTOM_LIST = BOTTOM_30_DATA["code_number"].values

for top in TOP_LIST : 
    
    # 종목 순환
    data[data["code"] == top]
    split_date = pd.Timestamp("2019-09-30")
    
    train = data.loc[:split_date, ["open", "high", "low", "close"]]
    test = data.loc[split_date:, ["open", "high", "low", "close"]]
    
    # MinmaxSacling
    from sklearn.preprocessing import MinMaxScaler
    sc = MinMaxScaler()
    train_sc = sc.fit_transform(train)
    test_sc = sc.transform(test)
    
    # Transform to Pandas DataFrame
    train_sc_df = pd.DataFrame(train_sc, columns=["open", "high", "low", "close"], index=train.index)
    test_sc_df = pd.DataFrame(test_sc, columns=["open", "high", "low", "close"], index=test.index)
    
    # List
    columns = ["open", "high", "low", "close"]

    # Setting for Sliding Window
    for col in columns : 
        for s in range(5, 17):
            train_sc_df['{} shift_-{}'.format(col,s)] = train_sc_df[col].shift(s)
            test_sc_df['{} shift_-{}'.format(col,s)] = test_sc_df[col].shift(s)
            
    # Nan Omit
    X_train = train_sc_df.dropna().drop(["open", "high", "low", "close"], axis=1)
    y_train = train_sc_df.dropna()[["open", "high", "low", "close"]]["close"]
    X_test = test_sc_df.dropna().drop(["open", "high", "low", "close"], axis=1)
    y_test = test_sc_df.dropna()[["open", "high", "low", "close"]]["close"]
    
    # ndarray to Transform
    X_train = X_train.values
    X_test = X_test.values
    y_train = y_train.values
    y_test = y_test.values
    
    # Final Training Set & Test Set's
    X_train_t = X_train.reshape(X_train.shape[0], 48, 1)
    X_test_t = X_test.reshape(X_test.shape[0], 48, 1)
    
    # Seed setting
    from numpy.random import seed
    seed(1234)
    from tensorflow import set_random_seed
    set_random_seed(1234)
    
    # LSTM
    from keras.layers import LSTM 
    from keras.models import Sequential 
    from keras.layers import Dense 
    import keras.backend as K
    from keras.callbacks import EarlyStopping
    from keras.layers import Dense, Dropout, Activation

    import keras.backend.tensorflow_backend as K
    with K.tf.device('/gpu:0'):

        # Sequeatial Model 
        K.clear_session() 
        model = Sequential()

        # 1층
        model.add(LSTM(10, input_shape=(48,1))) # (timestep, feature) 
        model.add(Dense(1)) # output = 1 

        # 결과
        model.compile(loss='mean_squared_error', optimizer='adam') 
        model.summary()
        
    # Model Fitting
    import keras.backend.tensorflow_backend as K
    with K.tf.device('/gpu:0'):
        early_stop = EarlyStopping(monitor='loss', patience=30, verbose=1)
        model.fit(X_train_t, y_train, epochs=150, batch_size=30, verbose=1, callbacks=[early_stop])
        
    # Pred
    y_pred = model.predict(X_test_t)
    PRED = pd.DataFrame(y_pred)
    PRED.columns = ["PRED"]

    # Actual
    REAL = pd.DataFrame(y_test)
    REAL.columns = ["REAL"]

    RESULT_TEST = pd.concat([REAL,PRED], axis=1)
    RESULT_TEST["date"] = test.index[16:]
    RESULT_TEST = RESULT_TEST.set_index("date")
    y_pred = model.predict(X_test_t)
    
    # RMSE
    forecast_error = y_test.reshape(-1,1) - y_pred
    SS, RMSE = 0, 0
    for i in forecast_error : 
        SS = SS + i * i
        RMSE = SS / len(forecast_error)
    print("{} DataSet's RMSE : {}" .format(top, float(RMSE**0.5)))

KeyError: "Cannot get right slice bound for non-unique label: Timestamp('2019-09-30 00:00:00')"